In [1]:
import torch
import numpy as np
import os
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

In [2]:
device="mps" if torch.backends.mps.is_available() else "cpu"
device

'mps'

In [3]:
n="60"
DATA_PATH=f"data/Embedings/Embeding-{n}"
TRAIN_EMBEDS = os.path.join(DATA_PATH, f'trainEmbeds{n}.npz')
TEST_EMBEDS = os.path.join(DATA_PATH, f'testEmbeds{n}.npz')
trainEmbeds, trainLabels = np.load(TRAIN_EMBEDS, allow_pickle=True).values()
testEmbeds, testLabels = np.load(TEST_EMBEDS, allow_pickle=True).values()
trainEmbeds.shape

(55144, 512)

In [4]:
import pickle
with open('data/class_list.pkl', 'rb') as file:
    ClassList = pickle.load(file)
print(len(ClassList))    
def name_from_index(i, ClassList=ClassList):
    reversed_dict = {v: k for k, v in ClassList.items()}
    if isinstance(i, np.ndarray):  # Check if i is a NumPy array
        names = [reversed_dict.get(idx.item(), None) for idx in i]
        return names
    else:
        return reversed_dict.get(i, None)

19


In [5]:
import torch.nn as nn
import torch.optim as optim
X = torch.tensor(trainEmbeds, dtype=torch.float32).to(device)
y = torch.tensor(trainLabels, dtype=torch.long).to(device)
X_test=torch.tensor(testEmbeds,dtype=torch.float32).to(device)
y_test=torch.tensor(testLabels,dtype=torch.long).to(device)

# Defining SVM model

In [13]:
class SoftmaxUsed(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(nn.Linear(512, 128),
                                     nn.ReLU(),
                                     nn.Dropout(0.6),
                                     nn.Linear(128, len(ClassList)),
                                     nn.LogSoftmax(dim=1))
    def forward(self, x):
        return self.layers(x)
model = SoftmaxUsed().to(device)
# model=SVM().to(device)
model.load_state_dict(torch.load(f="data/classLen- 19/embeding-60/predictFacesModelAcc96.45_With60Embeds.pth"))

<All keys matched successfully>

# Loss function and Optimizer function

In [14]:
# loss_func = nn.CrossEntropyLoss()
loss_func=nn.NLLLoss()
admoptimizer = optim.Adam(model.parameters(), lr=0.000051)

# Accuracy Function

In [15]:
def acc(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item() # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred)) * 100 
    return acc

# Training Loop

In [16]:
def modelSave(model):
    from pathlib import Path

    # 1. Create models directory 
    MODEL_PATH = Path(f"data/classLen- {len(ClassList)}/embeding-{n}")
    MODEL_PATH.mkdir(parents=True, exist_ok=True)

    # 2. Create model save path 
    MODEL_NAME = f"predictFacesModelAcc{testAcc:.2f}_With{n}Embeds.pth"
    MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

    # 3. Save the model state dict 
    print(f"Saving model to: {MODEL_SAVE_PATH}")
    torch.save(obj=model.state_dict(), # only saving the state_dict() only saves the models learned parameters
               f=MODEL_SAVE_PATH)

In [17]:
%%time
num_epochs = 2400000
trainingLoss= []
testingLoss =1
trainAcc = []
testAcc =0
for epoch in range(num_epochs):
    outputs = model(X)
    _, y_predict = torch.max(outputs, 1)
    train_accuracy = acc(y, y_predict)
    train_loss = loss_func(outputs, y)

    admoptimizer.zero_grad()
    train_loss.backward()
    admoptimizer.step()
    model.eval()
    with torch.no_grad():
        predictions = model(X_test)
        _, pred_labels = torch.max(predictions, 1)
        y_test = y_test.to(torch.long)  # Convert to Long data type
        test_accuracy = acc(y_test, pred_labels)
        test_loss = loss_func(predictions, y_test)
    if epoch % 10 == 0:
        if testAcc<test_accuracy:
            testAcc=test_accuracy
            if testAcc>96.5:
                modelSave(model)
        if test_loss<testingLoss and testAcc>90:
            testingLoss=test_loss-0.001
            modelSave(model)
            
        print(f"Epoch: {epoch} | TrainingLoss= {train_loss} | TestingLoss= {test_loss} | Accuracy= {test_accuracy}")


Saving model to: data/classLen- 19/embeding-60/predictFacesModelAcc96.10_With60Embeds.pth
Epoch: 0 | TrainingLoss= 19.156450271606445 | TestingLoss= 0.12106762826442719 | Accuracy= 96.09929078014184
Epoch: 10 | TrainingLoss= 0.25543397665023804 | TestingLoss= 0.12142343819141388 | Accuracy= 96.09929078014184
Epoch: 20 | TrainingLoss= 0.2550542652606964 | TestingLoss= 0.12047537416219711 | Accuracy= 96.45390070921985
Epoch: 30 | TrainingLoss= 0.2538135349750519 | TestingLoss= 0.1200864166021347 | Accuracy= 96.45390070921985
Epoch: 40 | TrainingLoss= 0.2526363730430603 | TestingLoss= 0.12022421509027481 | Accuracy= 96.45390070921985
Epoch: 50 | TrainingLoss= 0.2517208456993103 | TestingLoss= 0.12026605755090714 | Accuracy= 96.45390070921985
Saving model to: data/classLen- 19/embeding-60/predictFacesModelAcc96.45_With60Embeds.pth
Epoch: 60 | TrainingLoss= 0.25104397535324097 | TestingLoss= 0.12000381201505661 | Accuracy= 96.45390070921985
Epoch: 70 | TrainingLoss= 0.25054511427879333 | Te

Epoch: 710 | TrainingLoss= 0.24844317138195038 | TestingLoss= 0.11946319788694382 | Accuracy= 96.45390070921985
Epoch: 720 | TrainingLoss= 0.24844111502170563 | TestingLoss= 0.11942916363477707 | Accuracy= 96.45390070921985
Epoch: 730 | TrainingLoss= 0.24843889474868774 | TestingLoss= 0.11944159120321274 | Accuracy= 96.45390070921985
Epoch: 740 | TrainingLoss= 0.24843665957450867 | TestingLoss= 0.11942440271377563 | Accuracy= 96.45390070921985
Epoch: 750 | TrainingLoss= 0.24843467772006989 | TestingLoss= 0.1194269061088562 | Accuracy= 96.45390070921985
Epoch: 760 | TrainingLoss= 0.2484324425458908 | TestingLoss= 0.11942924559116364 | Accuracy= 96.45390070921985
Epoch: 770 | TrainingLoss= 0.24843046069145203 | TestingLoss= 0.1194302886724472 | Accuracy= 96.45390070921985
Epoch: 780 | TrainingLoss= 0.2484285831451416 | TestingLoss= 0.11943807452917099 | Accuracy= 96.45390070921985
Epoch: 790 | TrainingLoss= 0.24842649698257446 | TestingLoss= 0.11945535242557526 | Accuracy= 96.45390070921

Epoch: 1450 | TrainingLoss= 0.2482984960079193 | TestingLoss= 0.11918661743402481 | Accuracy= 96.45390070921985
Epoch: 1460 | TrainingLoss= 0.24829645454883575 | TestingLoss= 0.11918042600154877 | Accuracy= 96.45390070921985
Epoch: 1470 | TrainingLoss= 0.24829456210136414 | TestingLoss= 0.11920661479234695 | Accuracy= 96.45390070921985
Epoch: 1480 | TrainingLoss= 0.24829238653182983 | TestingLoss= 0.11915730684995651 | Accuracy= 96.45390070921985
Epoch: 1490 | TrainingLoss= 0.24829038977622986 | TestingLoss= 0.1191507875919342 | Accuracy= 96.45390070921985
Epoch: 1500 | TrainingLoss= 0.2482883781194687 | TestingLoss= 0.119167760014534 | Accuracy= 96.45390070921985
Epoch: 1510 | TrainingLoss= 0.24828630685806274 | TestingLoss= 0.11916052550077438 | Accuracy= 96.45390070921985
Epoch: 1520 | TrainingLoss= 0.24828413128852844 | TestingLoss= 0.11918547004461288 | Accuracy= 96.45390070921985
Epoch: 1530 | TrainingLoss= 0.2482820600271225 | TestingLoss= 0.1191333457827568 | Accuracy= 96.45390

Epoch: 2180 | TrainingLoss= 0.2481415867805481 | TestingLoss= 0.11889024823904037 | Accuracy= 96.45390070921985
Epoch: 2190 | TrainingLoss= 0.24813909828662872 | TestingLoss= 0.11888338625431061 | Accuracy= 96.45390070921985
Epoch: 2200 | TrainingLoss= 0.24813678860664368 | TestingLoss= 0.11893481016159058 | Accuracy= 96.45390070921985
Epoch: 2210 | TrainingLoss= 0.24813435971736908 | TestingLoss= 0.11890154331922531 | Accuracy= 96.45390070921985
Epoch: 2220 | TrainingLoss= 0.24813216924667358 | TestingLoss= 0.11887489259243011 | Accuracy= 96.45390070921985
Epoch: 2230 | TrainingLoss= 0.2481299191713333 | TestingLoss= 0.11892236024141312 | Accuracy= 96.45390070921985
Epoch: 2240 | TrainingLoss= 0.24812772870063782 | TestingLoss= 0.11887960135936737 | Accuracy= 96.45390070921985
Epoch: 2250 | TrainingLoss= 0.24812498688697815 | TestingLoss= 0.11888308078050613 | Accuracy= 96.45390070921985
Epoch: 2260 | TrainingLoss= 0.24812261760234833 | TestingLoss= 0.11885187029838562 | Accuracy= 96.

Epoch: 2910 | TrainingLoss= 0.2479497194290161 | TestingLoss= 0.11872551590204239 | Accuracy= 96.45390070921985
Epoch: 2920 | TrainingLoss= 0.24794720113277435 | TestingLoss= 0.11879836767911911 | Accuracy= 96.45390070921985
Epoch: 2930 | TrainingLoss= 0.24794459342956543 | TestingLoss= 0.11873575299978256 | Accuracy= 96.45390070921985
Epoch: 2940 | TrainingLoss= 0.24794159829616547 | TestingLoss= 0.11874225735664368 | Accuracy= 96.45390070921985
Epoch: 2950 | TrainingLoss= 0.24793916940689087 | TestingLoss= 0.11873222142457962 | Accuracy= 96.45390070921985
Epoch: 2960 | TrainingLoss= 0.24793648719787598 | TestingLoss= 0.11873526871204376 | Accuracy= 96.45390070921985
Epoch: 2970 | TrainingLoss= 0.2479330450296402 | TestingLoss= 0.11877580732107162 | Accuracy= 96.45390070921985
Epoch: 2980 | TrainingLoss= 0.2479304075241089 | TestingLoss= 0.11871911585330963 | Accuracy= 96.45390070921985
Epoch: 2990 | TrainingLoss= 0.24792735278606415 | TestingLoss= 0.11874473839998245 | Accuracy= 96.4

...

Epoch: 221670 | TrainingLoss= 0.17884474992752075 | TestingLoss= 0.13208086788654327 | Accuracy= 95.39007092198581
Epoch: 221680 | TrainingLoss= 0.1788501739501953 | TestingLoss= 0.13179974257946014 | Accuracy= 95.39007092198581
Epoch: 221690 | TrainingLoss= 0.1788477748632431 | TestingLoss= 0.13190430402755737 | Accuracy= 95.39007092198581
Epoch: 221700 | TrainingLoss= 0.17884138226509094 | TestingLoss= 0.13176296651363373 | Accuracy= 95.39007092198581
Epoch: 221710 | TrainingLoss= 0.1788395792245865 | TestingLoss= 0.13188327848911285 | Accuracy= 95.39007092198581
Epoch: 221720 | TrainingLoss= 0.1788388341665268 | TestingLoss= 0.13149437308311462 | Accuracy= 95.39007092198581
Epoch: 221730 | TrainingLoss= 0.17883466184139252 | TestingLoss= 0.1319652795791626 | Accuracy= 95.39007092198581
Epoch: 221740 | TrainingLoss= 0.17883171141147614 | TestingLoss= 0.131736159324646 | Accuracy= 95.39007092198581
Epoch: 221750 | TrainingLoss= 0.17882992327213287 | TestingLoss= 0.13193030655384064 | 

Epoch: 222390 | TrainingLoss= 0.17867927253246307 | TestingLoss= 0.1320788860321045 | Accuracy= 95.39007092198581
Epoch: 222400 | TrainingLoss= 0.1786792129278183 | TestingLoss= 0.13185454905033112 | Accuracy= 95.39007092198581
Epoch: 222410 | TrainingLoss= 0.17867712676525116 | TestingLoss= 0.13185307383537292 | Accuracy= 95.39007092198581
Epoch: 222420 | TrainingLoss= 0.17867428064346313 | TestingLoss= 0.13185647130012512 | Accuracy= 95.39007092198581
Epoch: 222430 | TrainingLoss= 0.1786731481552124 | TestingLoss= 0.13179729878902435 | Accuracy= 95.39007092198581
Epoch: 222440 | TrainingLoss= 0.17867201566696167 | TestingLoss= 0.13184306025505066 | Accuracy= 95.39007092198581
Epoch: 222450 | TrainingLoss= 0.178671196103096 | TestingLoss= 0.13183274865150452 | Accuracy= 95.39007092198581
Epoch: 222460 | TrainingLoss= 0.1786659210920334 | TestingLoss= 0.13175204396247864 | Accuracy= 95.39007092198581
Epoch: 222470 | TrainingLoss= 0.17866520583629608 | TestingLoss= 0.13176310062408447 |

Epoch: 223110 | TrainingLoss= 0.17851446568965912 | TestingLoss= 0.13178658485412598 | Accuracy= 95.39007092198581
Epoch: 223120 | TrainingLoss= 0.17851506173610687 | TestingLoss= 0.13155768811702728 | Accuracy= 95.39007092198581
Epoch: 223130 | TrainingLoss= 0.17851217091083527 | TestingLoss= 0.13160552084445953 | Accuracy= 95.39007092198581
Epoch: 223140 | TrainingLoss= 0.1785096377134323 | TestingLoss= 0.13161519169807434 | Accuracy= 95.39007092198581
Epoch: 223150 | TrainingLoss= 0.17850328981876373 | TestingLoss= 0.1316567212343216 | Accuracy= 95.39007092198581
Epoch: 223160 | TrainingLoss= 0.17850439250469208 | TestingLoss= 0.131736159324646 | Accuracy= 95.39007092198581
Epoch: 223170 | TrainingLoss= 0.17850501835346222 | TestingLoss= 0.1315714716911316 | Accuracy= 95.39007092198581
Epoch: 223180 | TrainingLoss= 0.1784973442554474 | TestingLoss= 0.13173167407512665 | Accuracy= 95.39007092198581
Epoch: 223190 | TrainingLoss= 0.1784932166337967 | TestingLoss= 0.13167144358158112 | 

Epoch: 223830 | TrainingLoss= 0.17834283411502838 | TestingLoss= 0.13091866672039032 | Accuracy= 95.39007092198581
Epoch: 223840 | TrainingLoss= 0.17834751307964325 | TestingLoss= 0.1311986893415451 | Accuracy= 95.39007092198581
Epoch: 223850 | TrainingLoss= 0.17834046483039856 | TestingLoss= 0.13096795976161957 | Accuracy= 95.39007092198581
Epoch: 223860 | TrainingLoss= 0.17833437025547028 | TestingLoss= 0.13102899491786957 | Accuracy= 95.39007092198581
Epoch: 223870 | TrainingLoss= 0.17833088338375092 | TestingLoss= 0.13089680671691895 | Accuracy= 95.39007092198581
Epoch: 223880 | TrainingLoss= 0.17833060026168823 | TestingLoss= 0.13111130893230438 | Accuracy= 95.39007092198581
Epoch: 223890 | TrainingLoss= 0.17833872139453888 | TestingLoss= 0.13095709681510925 | Accuracy= 95.39007092198581
Epoch: 223900 | TrainingLoss= 0.1783229559659958 | TestingLoss= 0.13109718263149261 | Accuracy= 95.39007092198581
Epoch: 223910 | TrainingLoss= 0.17832022905349731 | TestingLoss= 0.130961358547210

KeyboardInterrupt: 

In [18]:
from pathlib import Path

# 1. Create models directory 
MODEL_PATH = Path(f"data/classLen- {len(ClassList)}/embeding-{n}")
MODEL_PATH.mkdir(parents=True, exist_ok=True)

# 2. Create model save path 
MODEL_NAME = f"predictFacesModelAcc{test_accuracy:.2f}_With{n}Embeds.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

# 3. Save the model state dict 
print(f"Saving model to: {MODEL_SAVE_PATH}")
torch.save(obj=model.state_dict(), # only saving the state_dict() only saves the models learned parameters
           f=MODEL_SAVE_PATH)

Saving model to: data/classLen- 19/embeding-60/predictFacesModelAcc95.39_With60Embeds.pth
